-   [Parasitoids](#chap:c9)
    -   [Parasitoid-host dynamics](#parasitoid-host-dynamics)
    -   [Stability and resonant periodicity](#stability-and-resonant-periodicity)
    -   [Biological control](#biological-control)
    -   [Larch bud moth](#larch-bud-moth)
    -   [Host-parasitoid metapopulation dynamics](#host-parasitoid-metapopulation-dynamics)
    -   [ShinyApp](#shinyapp)

Parasitoids
===========

Parasitoid-host dynamics
------------------------

Many of the classic studies of the spatio-temporal dynamics of natural enemies and their hosts consider parasitoid-host interactions. Parasitoids represents a fascinating group of insect ‘infections’. Adults are free-living and lay their eggs in larvae (or eggs) of host insects. Hosts die when the parasitoid(s) complete their development and adults emerge from the infected hosts. From a dynamical systems point of view parasitoid-host interactions share many features of infectious disease dynamics. It is therefore instructive to cap our discussion of spatiotemporal dynamics with a discussion of this ecological interaction.

 conducted a cage experiment involving greenhouse white flies (*Trialeurodes vaporariorum*) and its parasitoid *Encarsia formosa*. The population was followed for 21 generations (fig. \[fig:burnett\]). The two populations oscillated in increasingly violent cycles until the parasitoid went extinct.

In [ ]:
data(burnett)
plot(burnett$Generation, 
   burnett$NumberofHostsParasitized, type="b", 
   ylab="Numbers", xlab="Generation")
lines(burnett$Generation, 
   burnett$NumberofHostsUnparasitized, type="b", 
   col=2, pch=2)
legend("topleft", legend=c("Parasitoid", "Host"), 
   lty=c(1,1), pch=c(1,2), col=c(1,2))

developed the first mathematical model for this interaction. Assuming random search (with a searching efficiency $a$) by the parasitoids, the probability of escaping parasitation is $\exp(-a P_t)$ and the number of host, $H$, and parasitoids, $P$, in the next generation is: $$\begin{aligned}
&H_{t+1} & = R H_t \exp(-a P_t)\\
&P_{t+1} & = R H_t (1-\exp(-a P_t)),\end{aligned}$$ where $R$ is the average number of offspring per hosts. An function for the host-parasitoid dynamics is:

In [ ]:
NB = function(R, a, T = 100, H0 = 10, P0 = 1){
   #T is length of simulation (number of time-steps)
   #H0 and P0 are initial numbers
   H=rep(NA, T) #Host series
   P=rep(NA, T) #Parasitoid series
   H[1] = H0 #Initiating the host series
   P[1] = P0 #Initiating the parasitoid series

   for(t in 2:T){
     H[t] = R * H[t-1] * exp(- a * P[t-1])
     P[t] = R * H[t-1] * (1-exp(- a * P[t-1]))
   } 

   res= list(H = H, P = P) 
   return(res)
} 

The Nicholson-Bailey models shows that with density-independent growth of the host (in the absence of parasitism) and random search by the parasitoid, predicts cycles with ever-increasing amplitude until the host and/or parasitoid goes extinct as seen in experiment. Let’s assume a host growth rate $R$ of 1.1 and a parasitoid searching efficiency $a$ of 0.1. We use the `NB()`-function to simulate the Nicholson-Bailey model and plot host/parasitoid abundance against time, and host-parasitoids in the phase plane (fig. \[fig:nb\])

In [ ]:
sim = NB(R = 1.1, a = 0.1)
time = 1:100
par(mfrow=c(1,2))
plot(time, sim$H, type =  "l", xlab  =  "Generations", 
     ylab  =  "Host abundance", ylim = c(0,14))
points(time, sim$P, type = "l", col = "red")
plot(sim$H,sim$P, type = "l", xlab = "Host abundance", 
     ylab = "Parasitoid abundance")

We can assume a sequence of searching efficiencies between 0 and 1 to explore how the time to extinction of the parasitoid depend on the searching efficiency. We make use of the functions `which` and `min` to store the time to extinction. Persistence time is greatest at intermediate search efficiency (fig. \[fig:tte\]).

In [ ]:
aVals = seq(0,1,by = 0.01)
tte = rep(NA,length(aVals))
for (i in c(1:length(aVals))){
     sim =  NB(R = 1.1, a = aVals[i], T = 500)
     tte[i] = min(which(sim$P==0))
     }
plot(aVals,tte,type = "b", ylab="TTE", 
     xlab="Search efficiency")

suggested that $R=2$ and $a=0.067$ was appropriate values for this system. Let’s check if we agree by minimizing sum-of-square-errors between observed and predicted abundances. We estimate $R$ and $a$ on a log-scale to make sure they are strictly positive quantities.

In [ ]:
ssfn=function(par){
  R=exp(par[1])
  a=exp(par[2])
  sim=NB(R,a, T=22, H0=10.1, P0=11.9)
  ss=sum((burnett$NumberofHostsUnparasitized-sim$H)^2+
      (burnett$NumberofHostsParasitized-sim$P)^2)
 return(ss)
 }
par=log(c(2, 0.05)) 
fit=optim(par, ssfn)
exp(fit$par)

Our fit is close to Burnett’s. Figure \[fig:burnett2\] shows the model prediction using our best-guess parameters.

In [ ]:
sim = NB(R=2.16767, a=0.06812, T=22, H0=10.1, P0=11.9)
plot(burnett$Generation, 
   burnett$NumberofHostsParasitized, type="b", 
   ylab="Numbers", xlab="Generation")
lines(burnett$Generation, sim$P)
lines(burnett$Generation, 
   burnett$NumberofHostsUnparasitized, type="b", 
   col=2, pch=2)
lines(burnett$Generation, sim$H, col=2)
legend("topleft", legend=c("Parasitoid", "NB-P", 
   "Host", "NB-H"), lty=c(1,1,1,1), pch=c(1,NA,2,NA),
   col=c(1,1,2,2))

Stability and resonant periodicity
----------------------------------

 did a detailed mathematical analysis of the model and showed that the equilibrium is an *unstable focus* regardless of parameter values. We can revisit on the concepts from chapter \[chap:c10\] for the model. The equilibrium of the Nicholson-Bailey model is $P^* = log(R)/a$, $H^*= log(R) /(a (R-1))$. The eigenvalues of the Jacobian evaluated at the equilibrium are:

In [ ]:
F = expression(R*H*exp(-a * P))
G = expression(R*H*(1-exp(-a * P)))
j11 = D(F, "H"); j12 = D(F, "P") 
j21 = D(G, "H"); j22 = D(G, "P") 
R = 2.17; a = 0.068
params = c(R = R, a = a, P = log(R)/a, H =  log(R)/(a*(R-1)))
J = with(as.list(params),
    matrix(c(eval(j11), eval(j12), eval(j21), 
    eval(j22)), ncol = 2, byrow = TRUE))
eigen(J, only.values = TRUE)$values
max(abs(eigen(J)$values))

It is an unstable focus since the eigenvalues are a pair of complex conjugates whose absolute value is greater than one[1]. Since this is a difference model, the predicted period of the outwards spiral is $2 \pi / \tan^{-1}(\frac{\mbox{Im}(\lambda)}{\mbox{Re}(\lambda)})$:

In [ ]:
2*pi/atan2(Im(eigen(J)$values[1]), 
     Re(eigen(J)$values[1]))

Let us explore how the cycle period depend on host growth rate (fig. \[fig:hpper\]):

In [ ]:
RVals = seq(1.1, 3, by = 0.1)
per = rep(NA, length(RVals))
for(i in 1:length(RVals)){
R = RVals[i]
a = 0.068
params = c(R = R, a = a, P = log(R)/a,
     H =  log(R)/(a*(R-1)))
J = with(as.list(params),
    matrix(c(eval(j11), eval(j12), eval(j21), 
    eval(j22)), ncol = 2, byrow = TRUE))
per[i] = 2*pi/atan2(Im(eigen(J)$values[1]), 
     Re(eigen(J)$values[1]))
}
plot(RVals, per, type = "b", xlab = "R", ylab = "Period")

The higher the host growth rate, the faster the outwards spiral.

Biological control
------------------

Parasitoids has been used for bio-control of agricultural pest through the ages . Successful bio-control require that the natural enemy – in this case parasitoid – keeps the pest consistently below an economic threshold. The inherent instability predicted by the Nicholson-Bailey model is at odds with successful biocontrol by parasitoids. Many different model modifications have been analyzed to see when stable regulation can happen. These include: (i) long-lived adult hosts, (ii) density-dependent host growth, (iii) heterogeneity in risk such as aggregated attack rates, spatial heterogeneity, host refugia and (iv) interference among parasitoids . showed – by replacing the Poisson-attack assumption with a negative binomial distribution – how heterogeneity in risk stabilizes dynamics. He coined the CV$^2$-rule which says that if the coefficient-of-variation in attack rate is greater than 1, the parasitoid-host dynamics stabilizes. model is: $$\begin{aligned}
H_{t+1} & = R H_t (1+\frac{a P_t}{k})^{-k}\\
P_{t+1} & = R H_t (1-(1+\frac{a P_t}{k})^{-k}).\end{aligned}$$ A Shiny-app of the negative binomial model, `May.app`, can be found in the `epimdr`-package.

As with infectious diseases, parasitoids-host interactions persist regionally even with local non-persistence through regional consumer-resource metapopulation dynamics.

Larch bud moth
--------------

Parasitoids cause violent fluctuations in the dynamics of the larch bud moth across the European Alps. Historical records shows recurrent traveling waves of defoliation every 9 years for centuries [2]. developed a model of the interactions among the larch bud worm, its parasitoids and the host plant. showed that a spatial extension of this model predicts the observed waves. However, for the more general purpose of considering spatiotemporal host-parasitoid dynamics we will use the simpler spatially-extended Nicholson-Bailey model.

Host-parasitoid metapopulation dynamics
---------------------------------------

Coupled map lattice models have a long history in the study of parasitoid-host dynamics . highlighted the importance of allowing for different mobility of the host and parasitoid. We define `Dh` as the proportion of host that disperses to neighboring patches, and `Dp` the proportion of parasitoid that disperses. showed that changing these can shift the spatial dynamics between spatial chaos, spiral waves or stable spatial heterogeneity. We construct a Nicholson-Bailey CML along the lines introduced in chapter \[chap:c8\]:

In [ ]:
#Dh is proportion of hosts that disperses 
#Dp is proportion of parasitoids that disperses
Dh = 0.5
Dp = 0.7
#xlen is width of the lattice (E-W)
#ylen is height of the lattice (N-S)
xlen = 30
ylen = 30

The `hp.dyn`-function defines the function to update the local abundances of hosts and parasitoids according to the Nicholson-Bailey model. Previous densities of host, `h`, and parasitoids, `p`, need to be supplied as arguments to the function, in addition to the host growth rate (`R`) and parasitoid search efficiency `a`.

In [ ]:
hp.dyn = function(h, p, R, a) {
    # hnew is the post-interaction host density
    hnew = R * h * exp(-a * p)
    # pnew is the post-interaction parasitoid density
    pnew = R * h * (1 - exp(-a * p))
    # the two vectors of results are stored in a 'list'
    res = list(h = hnew, p = pnew)
    return(res)
}

We generate spatial coordinates and calculate the distance between all populations:

In [ ]:
xy = expand.grid(1:xlen, 1:ylen)
dmat = as.matrix(dist(xy))

The redistribution matrix is calculated by checking if the distance in `dmat` is smaller than 1.5, thus flagging all populations that are first neighbors. If the distance is $<1.5$ we assign a value of `Dh/8` and `Dp/8`. The fractions that do not disperse (`1-Dh` and `1-Dp`) are along the diagonal of the redistribution matrices.

In [ ]:
kh = ifelse(dmat < 1.5, Dh/8, 0)
kp = ifelse(dmat < 1.5, Dp/8, 0)
diag(kh) = 1 - Dh
diag(kp) = 1 - Dp

We finally construct matrices to store results and set starting conditions for the simulation. `IT` is number of iterations. The initial conditions are zeros everywhere (i.e., the first column in each matrix gets zeros), except for an arbitrary population (in this case 23) which starts with 4 hosts and 1 parasitoid.

In [ ]:
IT = 600
hmat = matrix(NA, nrow = xlen * ylen, ncol = IT)
pmat = matrix(NA, nrow = xlen * ylen, ncol = IT)
hmat[, 1] = 0
pmat[, 1] = 0
hmat[23, 1] = 4
pmat[23, 1] = 1

We simulate from generation 2 to `IT`, storing the results on the way. We first simulate growth, using the function `hp.dyn`. Next, we redistribute the host parasitoids according to their dispersal matrices. The vector of pre-dispersal hosts, `tmp$h`, is redistributed through matrix multiplying the vector by the redistribution matrix, `kh`. The same is done for the parasitoid. The function `cat()` provides a progress monitor.

In [ ]:
for(i in 2:IT){
   #growth
   tmp = hp.dyn(h = hmat[,(i-1)], p = pmat[,(i-1)], 
      R = 2, a = 1)
   #redistribution
   hmat[,i] = tmp$h%*%kh;
   pmat[,i] = tmp$p%*%kp;
   cat(i, " of ", IT, "\r")
}

Finally, we are ready to plot our results. Here is the code to make an animation of the last 100 generations for the parasitoid:

In [ ]:
#plot the last 100 generations for the parasitoid
for(i in 1:100){
   x=xy[,1]
   y=xy[,2]
   z=pmat[,i+500]
   symbols(x,y, fg=2, circles=z, inches=0.1, 
      bg=2, xlab="", ylab="")
   Sys.sleep(.1) #this is to slow down the plotting
}

and the code to write frames to the disk to make a permanent web-optimized movie:

In [ ]:
for (k in 1:50) {
    png(filename = paste("Pplot", k, ".png", sep = ""))
    x = xy[, 1]
    y = xy[, 2]
    z = pmat[, k + 500]
    symbols(x, y, fg = 2, circles = z, inches = 0.1, bg = 2, 
        xlab = "", ylab = "")
    dev.off()
}
system("convert P*.png -delay 500 -coalesce -layers OptimizeTransparency cml2.gif")
system("rm *.png")

Low mobility of both host and parasitoid (`Dp=Dh=0.1`) leads to spatially chaotic dynamics and high mobility (`Dh=0.5, Dp=0.7`) leads to spiral waves . Animated gifs of the two dynamic regimes are on:
<https://github.com/objornstad/epimdr/blob/master/mov/cml1.gif> and
<https://github.com/objornstad/epimdr/blob/master/mov/cml2.gif>.

ShinyApp
--------

The `epimdr`-package contains a Shiny-app to study the negative-binomial parasitoid-host model. The app can be launched through:

In [ ]:
require(shiny)
May.app

[1] Recall that according to [](https://en.wikipedia.org/wiki/Stability_theory), stability of discrete time models requires the absolute value of the largest eigenvalue of the Jacobian evaluated at the equilibrium to be smaller than 1 – as opposed to continuous time models for which the requirement is that the real part must be smaller than 0.

[2] <https://github.com/objornstad/epimdr/blob/master/mov/lbm.gif> shows an animated gif of Larch bud moth defoliation between 1960 and 2000.